HOW TO IMPLEMENT BACKTESTING USING BACKTRADER FOR THE PAST 20 YEARS IT CAN BE MORE

In [6]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

# For datetime objects
import datetime 

# To manage paths
import os.path 

# To find out the script name
import sys  

# Import the backtrader platform
import backtrader as bt
# Used for graphical analysis
import pandas as pd

# Import numpy for handling infinite values in the dataframe
import numpy as np  

# Import matplotlib for interactive plotting
import matplotlib.pyplot as plt

 # Switch to an interactive backend that is cerebo
plt.switch_backend('tkagg') 

# Create a Stratey
class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
    )

    def log(self, txt, dt=None):

        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):

        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)

        # Indicators for the plotting show and displaying graphs
        bt.indicators.ExponentialMovingAverage(self.datas[0], period=25)
        bt.indicators.WeightedMovingAverage(self.datas[0], period=25,
                                            subplot=True)
        bt.indicators.StochasticSlow(self.datas[0])
        bt.indicators.MACDHisto(self.datas[0])
        rsi = bt.indicators.RSI(self.datas[0])
        bt.indicators.SmoothedMovingAverage(rsi, period=10)
        bt.indicators.ATR(self.datas[0], plot=False)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:

            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
                
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):

        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] > self.sma[0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.dataclose[0] < self.sma[0]:

                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

# This the main function that performs the effectiveness of the strategy
                
if __name__ == '__main__':

    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    cerebro.addstrategy(TestStrategy)

    # Load historical data for forex pairs in this case EURUSD for the past 20 years
    df = pd.read_csv('historical_forex_data.csv', index_col='date', parse_dates=True)

    # Check for infinite values
    if df.isin([np.inf, -np.inf]).any().any():
        print("DataFrame contains infinite values. Cleaning...")
        
        # Replace infinite values with NaN
        df.replace([np.inf, -np.inf], np.nan, inplace=True)

        # Drop rows with NaN values
        df.dropna(inplace=True)

    # Sort DataFrame index
    df.sort_index(inplace=True)

    # Set the start and end dates
    startDate = datetime.datetime(2005, 1, 1)
    endDate = datetime.datetime(2024, 12, 31)

    # Filter data based on start and end dates
    df = df.loc[startDate:endDate]

    # Create a Data Feed
    data = bt.feeds.PandasData(dataname=df)

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100.0)

    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=10)

    # Set the commission
    cerebro.broker.setcommission(commission=0.0)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything that includes the logic
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Plot the results
    cerebro.plot(style='bar', iplot=False, volume=True)

    # Save plot to a file
    plt.savefig('plot.png')

  # Plot the time series with labels for years and prices
    plt.figure()
    plt.plot(df.index, df['close'], label='Close Price')
    plt.xlabel('Year')
    plt.ylabel('Price')
    plt.title('FOREX EURUSD')
    plt.legend()
    plt.show()

Starting Portfolio Value: 100.00
2005-04-25, Close, 1.30
2005-04-25, BUY CREATE, 1.30
2005-04-26, BUY EXECUTED, Price: 1.30, Cost: 12.99, Comm 0.00
2005-04-26, Close, 1.30
2005-04-27, Close, 1.29
2005-04-27, SELL CREATE, 1.29
2005-04-28, SELL EXECUTED, Price: 1.29, Cost: 12.99, Comm 0.00
2005-04-28, OPERATION PROFIT, GROSS -0.06, NET -0.06
2005-04-28, Close, 1.29
2005-04-29, Close, 1.29
2005-05-02, Close, 1.29
2005-05-03, Close, 1.29
2005-05-04, Close, 1.29
2005-05-05, Close, 1.30
2005-05-06, Close, 1.28
2005-05-09, Close, 1.28
2005-05-10, Close, 1.29
2005-05-11, Close, 1.28
2005-05-12, Close, 1.27
2005-05-13, Close, 1.26
2005-05-16, Close, 1.26
2005-05-17, Close, 1.26
2005-05-18, Close, 1.27
2005-05-19, Close, 1.26
2005-05-20, Close, 1.26
2005-05-23, Close, 1.26
2005-05-24, Close, 1.26
2005-05-25, Close, 1.26
2005-05-26, Close, 1.25
2005-05-27, Close, 1.26
2005-05-30, Close, 1.25
2005-05-31, Close, 1.23
2005-06-01, Close, 1.22
2005-06-02, Close, 1.23
2005-06-03, Close, 1.22
2005-06-06